# IBM AI Enterprise Workflow Capstone

## Part 2: Model Building & Selection

## Table of Contents
* [Feature Engineering](#first-bullet)
* [Update Log Files](#second-bullet)
* [Model Selection](#third-bullet)

## Feature Engineering <a class="anchor" id="first-bullet"></a>

Take in the time series data from the data ingestion and convert to a dictionary with keys for each country and features that describe revenue from past weeks and the previous year. 

In [1]:
%%writefile feature_engineering.py

import os
import re
import time
import numpy as np
import pandas as pd
from datetime import datetime
from collections import defaultdict

from data_ingestion import fetch_ts


def engineer_features(df, training = True):

    '''
    convert data into dictionary and adding features for previous days revenue
    '''

    dates = df['date'].values.copy()
    dates = dates.astype('datetime64[D]')

    eng_features = defaultdict(list)
    previous = [7, 14, 28, 70]
    y = np.zeros(dates.size)
    
    for d, day in enumerate(dates):

        for num in previous:
            
            current = np.datetime64(day, 'D') 
            prev = current - np.timedelta64(num, 'D')
            
            mask = np.in1d(dates, np.arange(prev, current, dtype = 'datetime64[D]'))
            eng_features['previous_{}'.format(num)].append(df[mask]['revenue'].sum())
 
        plus_30 = current + np.timedelta64(30, 'D')
    
        mask = np.in1d(dates, np.arange(current, plus_30, dtype = 'datetime64[D]'))
        y[d] = df[mask]['revenue'].sum()

        start_date = current - np.timedelta64(365, 'D')
        stop_date = plus_30 - np.timedelta64(365, 'D')
        
        mask = np.in1d(dates, np.arange(start_date, stop_date, dtype = 'datetime64[D]'))
        eng_features['previous_year'].append(df[mask]['revenue'].sum())

        minus_30 = current - np.timedelta64(30, 'D')
        
        mask = np.in1d(dates, np.arange(minus_30, current,dtype = 'datetime64[D]'))
        eng_features['recent_invoices'].append(df[mask]['unique_invoices'].mean())
        eng_features['recent_views'].append(df[mask]['total_views'].mean())

    X = pd.DataFrame(eng_features)
    X.fillna(0, inplace = True)
    
    mask = X.sum(axis = 1) > 0
    
    X = X[mask]
    y = y[mask]
    
    dates = dates[mask]
    
    X.reset_index(drop = True, inplace = True)

    if training == True:
        
        mask = np.arange(X.shape[0]) < np.arange(X.shape[0])[-30]
        X = X[mask]
        y = y[mask]
        dates = dates[mask]
        X.reset_index(drop = True, inplace = True)
    
    return(X, y, dates)


if __name__ == '__main__':
    
    run_start = time.time()
    
    data_dir = os.path.join('.', 'data', 'cs-train')
    df = fetch_ts(data_dir)
    
    m, s = divmod(time.time() - run_start, 60)
    h, m = divmod(m, 60)
    
    print('run time:', '%d:%02d:%02d'%(h, m, s))
    print('feature engineering complete')

Overwriting feature_engineering.py


In [2]:
%run feature_engineering.py

run time: 0:00:00
feature engineering complete


## Update Log Files <a class="anchor" id="second-bullet"></a>

- Update the train log with the timestamp, data shape, evaluation metric, and runtime. 
- Update the predict log with the timestamp, predicted value, probability value, and runtime. 

In [3]:
%%writefile logger.py

import time, os, re, csv, sys, uuid, joblib
from datetime import date

if not os.path.exists(os.path.join('.', 'logs')):
    os.mkdir('logs')
    
    
def update_train_log(data_shape, eval_test, runtime, model_vers, test = False):
    
    '''
    update train log file
    '''
    
    today = date.today()
    
    if test:
        logfile = os.path.join('logs', 'train-test.log')
    else:
        logfile = os.path.join('logs', 'train-{}-{}.log'.format(today.year, today.month))
        
    header = ['unique_id', 'timestamp', 'x_shape', 'eval_test', 'model_version', 'runtime']
    
    write_header = False
    
    if not os.path.exists(logfile):
        write_header = True
        
    with open(logfile, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter = ',')
        
        if write_header:
            writer.writerow(header)
            
        to_write = map(str, [uuid.uuid4(), time.time(), data_shape, eval_test, model_vers, runtime])
        writer.writerow(to_write)   
    
    
def update_predict_log(y_pred, y_proba, query, runtime, model_vers, test = False):
    
    '''
    update predict log file
    '''
    
    today = date.today()
    
    if test:
        logfile = os.path.join('logs', 'predict-test.log')
    else:
        logfile = os.path.join('logs', 'predict-{}-{}.log'.format(today.year, today.month))
        
    header = ['unique_id', 'timestamp', 'y_pred', 'y_proba', 'query', 'model_version', 'runtime']
    
    write_header = False
    
    if not os.path.exists(logfile):
        write_header = True
    
    with open(logfile, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter = ',')
        
        if write_header:
            writer.writerow(header)

        to_write = map(str, [uuid.uuid4(), time.time(), y_pred, y_proba, query, model_vers, runtime])
        writer.writerow(to_write)
        
        
if __name__ == '__main__':

    from model import model_vers
    
    update_train_log(str((100,10)), "{'rmse':0.5}", "00:00:01",
                     model_vers, test = True)

    update_predict_log('[0]', '[0.6, 0.4]', '["united_states", 24, "aavail_basic", 8]', '00:00:01',
                       model_vers, test = True)        


Overwriting logger.py


In [4]:
%run logger.py

## Model Selection <a class="anchor" id="third-bullet"></a>

Loop through models and choose the optimal model for each country based off RMSE. Train each country's data with the chosen model, load the model, and then predict the revenue given a country and date.

As our business opportunity revolves around predicting a numeric value for future revenue, we will iterate through different regressors and output their RMSE values to determine the best model. 

Suite of Models:
- Random Forest Regressor
- Ada Boost Regressor
- Gradient Boosting Regressor

Adjusted Parameters:
- Number of Estimators
- Learning Rate
- Maximum Number of Features

Hyperparameters were tuned for each algorithm with the use of scikit-learn classes such as `Pipeline` and `GridSearchCV`. 

In [5]:
%%writefile model.py

import os
import re
import time
import numpy as np
import pandas as pd
from datetime import datetime
from collections import defaultdict
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib as joblib

from data_ingestion import fetch_ts
from feature_engineering import engineer_features
from logger import update_predict_log, update_train_log

model_dir = os.path.join('models')
model_vers = 0.1


def _model_train(df, tag, test = False):
    
    '''
    loop through and train different models
    '''
    
    time_start = time.time()
    
    X, y, dates = engineer_features(df)
    
    if test:
        n_samples = int(np.round(0.3 * X.shape[0]))
        subset_indices = np.random.choice(np.arange(X.shape[0]), n_samples,
                                          replace = False).astype(int)
        mask = np.in1d(np.arange(y.size), subset_indices)
        y = y[mask]
        X = X[mask]
        dates = dates[mask]
        
    X, y, dates = engineer_features(df)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 100)
    
    reg_names = ['RF', 'ADA', 'GB']
    regressors = (RandomForestRegressor(random_state = 100), AdaBoostRegressor(random_state = 100), 
                  GradientBoostingRegressor(random_state = 100))

    params = [
        {'reg__n_estimators': [10, 15, 20, 25],
         'reg__max_features': [3, 4, 5]
        },
        {'reg__n_estimators': [10, 15, 20, 25],
         'reg__learning_rate': [1, 0.1, 0.01, 0.001]
        },
        {'reg__n_estimators': [10, 15, 20, 25],
         'reg__max_features': [3, 4, 5]
        }
    ]

    models = {}
    
    for iteration, (name, regressor, param) in enumerate(zip(reg_names, regressors, params)):
        
        pipeline = Pipeline(steps = [
            ('scaler', StandardScaler()),
            ('reg', regressor)
        ])
        
        grid = GridSearchCV(pipeline, param_grid = param, scoring = 'neg_mean_squared_error', cv = 5, 
                           n_jobs = -1, return_train_score = True)
        
        grid.fit(X_train, y_train)
        
        models[name] = grid, grid.best_estimator_.get_params()
        
    test_scores = []
    
    for key, model in models.items():
        
        y_pred = model[0].predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_pred, y_test))
        test_scores.append(rmse)
        
    best_model = reg_names[np.argmin(test_scores)]
    opt_model, params = models[best_model]
    
    country_name = tag.replace('_', ' ').title()
    
    print('Model Results for {}: \n'.format(country_name))
    
    print('RMSE Values:')
    
    for i in range(len(reg_names)):
        print('{}: {} \n'.format(reg_names[i], test_scores[i]))

    print('Best Model: \n {}'.format(next(iter(models.items()))[1][1]['reg']))
    
    print('============================================================')
    
    if test:
        saved_model = os.path.join(model_dir, 'test-{}-model-{}.joblib'.format(tag, re.sub('\.', '_', str(model_vers))))
    else:
        saved_model = os.path.join(model_dir, 'prod-{}-model-{}.joblib'.format(tag, re.sub('\.', '_', str(model_vers))))
                                   
    joblib.dump(opt_model, saved_model)
    
    m, s = divmod(time.time() - time_start, 60)
    h, m = divmod(m, 60)
    runtime = '%03d:%02d:%02d'%(h, m, s)

    update_train_log((str(dates[0]), str(dates[-1])), {'rmse': max(test_scores)}, runtime, model_vers, test = True)
    
    
def model_train(data_dir, test = False):

    '''
    train models for each country and select optimal model
    '''
    
    if not os.path.isdir(model_dir):
        os.mkdir(model_dir)
        
    ts_data = fetch_ts(data_dir)

    for country, df in ts_data.items():
        _model_train(df, country, test = test) 
        
        
def model_load(data_dir = None, training = True):

    '''
    load trained model
    '''

    if not data_dir:
        data_dir = os.path.join('.', 'data', 'cs-train')
    
    models = [f for f in os.listdir(os.path.join('.', 'models')) if f.endswith('.joblib')]

    if len(models) == 0:
        raise Exception('model cannot be found')

    all_models = {}
    
    for model in models:
        all_models[re.split('-', model)[1]] = joblib.load(os.path.join('.', 'models', model))

    ts_data = fetch_ts(data_dir)
    
    all_data = {}
    
    for country, df in ts_data.items():
        X, y, dates = engineer_features(df, training = training)
        dates = np.array([str(d) for d in dates])
        all_data[country] = {'X':X, 'y':y, 'dates': dates}
        
    return(all_data, all_models)


def model_predict(country, year, month, day, all_models = None, test = False):
    
    '''
    predict from model given country and date
    '''
    
    time_start = time.time()
    
    if not all_models:
        all_data, all_models = model_load(training = False)
        
    if country not in all_models.keys():
        raise Exception('model for country {} could not be found'.format(country))

    for d in [year, month, day]:
        if re.search('\D', d):
            raise Exception('invalid year, month, or day')

    model = all_models[country]
    data = all_data[country]
    
    target_date = '{}-{}-{}'.format(year, str(month).zfill(2), str(day).zfill(2))
    print(target_date)
    
    if target_date not in data['dates']:
        raise Exception('date {} not in range {}-{}'.format(target_date, data['dates'][0], data['dates'][-1]))
        
    date_indx = np.where(data['dates'] == target_date)[0][0]
    
    query = data['X'].iloc[[date_indx]]
    
    if data['dates'].shape[0] != data['X'].shape[0]:
        raise Exception('dimensions mismatch')
        
    y_pred = model.predict(query)
    y_proba = None
    
    if 'predict_proba' in dir(model) and 'probability' in dir(model):
        if model.probability == True:
            y_proba = model.predict_proba(query)
            
    print('y_pred: {}, y_proba: {}'.format(y_pred, y_proba))
     
    m, s = divmod(time.time() - time_start, 60)
    h, m = divmod(m, 60)
    runtime = '%03d:%02d:%02d'%(h, m, s)

    update_predict_log(y_pred, y_proba, target_date, runtime, model_vers, test = test)
    
    return({'y_pred': y_pred, 'y_proba': y_proba})
        
      
if __name__ == '__main__':

    data_dir = os.path.join('.', 'data', 'cs-train')
    model_train(data_dir, test = True)

    all_data, all_models = model_load()
    print('Models Loaded: ',', '.join(all_models.keys()))

    country = 'all'
    year = '2018'
    month = '01'
    day = '05'
    
    result = model_predict(country, year, month, day)
    print(result)   
    

Overwriting model.py


In [6]:
%run model.py

Model Results for Portugal: 

RMSE Values:
RF: 481.4510586410663 

ADA: 633.5492860870743 

GB: 680.0064118013312 

Best Model: 
 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=5, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=100, verbose=0, warm_start=False)
Model Results for Belgium: 

RMSE Values:
RF: 98.22937829428255 

ADA: 288.44677520183154 

GB: 296.7943232262412 

Best Model: 
 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=4, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=100, verbose=0, warm_start=False)
Model Results for United Kingdom: 

RMSE Values:
RF: 17626.2